## Ensure a Minikube cluster is running

- If you created a Minikube cluster with the command `minikube start --nodes 3` and it is still running:
  - Do nothing
- If you have a stopped (not deleted) Minikube cluster that was created with the command `minikube start --nodes 3`:
  - Run the command `minikube start`
- If you don't have a Minikube cluster (or deleted your previous one):
  - Run the command `minikube start --nodes 3`  

## Enable the Metallb addon

In [1]:
!minikube addons enable metallb

❗  metallb is a 3rd party addon and is not maintained or verified by minikube maintainers, enable at your own risk.
❗  metallb does not currently have an associated maintainer.
    ▪ Using image quay.io/metallb/speaker:v0.9.6
    ▪ Using image quay.io/metallb/controller:v0.9.6
🌟  The 'metallb' addon is enabled


## List Nodes

In [2]:
!kubectl get nodes -o wide

NAME           STATUS   ROLES           AGE   VERSION   INTERNAL-IP    EXTERNAL-IP   OS-IMAGE             KERNEL-VERSION                       CONTAINER-RUNTIME
minikube       Ready    control-plane   11h   v1.28.3   192.168.49.2   <none>        Ubuntu 22.04.3 LTS   5.15.133.1-microsoft-standard-WSL2   docker://24.0.7
minikube-m02   Ready    <none>          13m   v1.28.3   192.168.49.3   <none>        Ubuntu 22.04.3 LTS   5.15.133.1-microsoft-standard-WSL2   docker://24.0.7
minikube-m03   Ready    <none>          12m   v1.28.3   192.168.49.4   <none>        Ubuntu 22.04.3 LTS   5.15.133.1-microsoft-standard-WSL2   docker://24.0.7


## Get the Metallb ConfigMap YAML

- This will extract the Metallb ConfigMap from the cluster and store it in your file system under `manifests/configmap.yaml`.

In [3]:
!kubectl get cm -n metallb-system -o yaml > manifests/configmap.yaml
!kubectl get configmap -n metallb-system -o yaml > manifests/configmap.yaml

## Edit the Metallb ConfigMap

- Open the file `manifests/configmap.yaml`.
- Add an IP address range for `addresses` under `items.config.address-pools`.
  - Look at your nodes' `INTERNAL-IP` addreses above and choose an IP address range outside this range (but with teh same CIDR).
    - In my case, teh node's IP addressses were `192.168.49.2-192.168.49.4`
      - So I chose the range `192.168.49.200-192.168.49.220` for the `addresses` setting below.
  - Minikube will assign an avaialbe IP address from this range to any Service with `type: LoadBalancer`.
- Save the file.

```bash
items:
- apiVersion: v1
  data:
    config: |
      address-pools:
      - name: default
        protocol: layer2
        addresses:
        - 192.168.49.200-192.168.49.220  # add an IP range for the Metallb LoadBalancer here
```

## Deploy the updated Metallb ConfigMap

In [4]:
!kubectl apply -f manifests/configmap.yaml

configmap/config configured
configmap/kube-root-ca.crt configured


## Deploy a Deployment

- The Deployment definition is in the YAML file `manifests/deploy-app.yaml`.

In [5]:
!kubectl apply -f manifests/deploy-app.yaml

deployment.apps/deploy-nginx created


## Let's look at the Deployment's YAML

**Note:**

- The number of `replicas` is set to 3.
- The `revisionHistoryLimit` is set to 3.
- The Pod template's container is listening on `containerPort` 80.
- The Pod template's `labels` are `app: nginx` and `env: prod`
  - The Deployment's `matchLabels` match these labels
    - Therefore, the Deployment will create 3 replicas from the Pod template.
  - The Service's `selector` matches these labels
    - Therefore, the Service will load balance between the three Pod replicas.

```bash
apiVersion: apps/v1
kind: Deployment
metadata:
  name: deploy-nginx         # the deployment is named deploy-nginx
spec:
  replicas: 3                 # the deployment defines three replicas
  revisionHistoryLimit: 3     # the deployment defines a maximum of three ReplicaSet revisions to store in history
  selector:
    matchLabels:
      app: nginx              # the deployment has two matchLabels that match the Pod template's labels below
      env: prod               # app = nginx and env = prod
  template:
    metadata:
      name: myapp-pod
      labels:
        app: nginx             # the Pod template has two labels defined (the Service's selector matches these two labels):
        env: prod              # app = nginx and env = prod
    spec:
      containers:
      - name: nginx
        image: nginx:alpine    # the container is based on the nginx:alpine image
        resources:
          requests:
            cpu: 100m
            memory: 128Mi
          limits:
            cpu: 250m
            memory: 256Mi    
        ports:
        - containerPort: 80    # the Pod template's container is listening on containerPort 80
```

In [6]:
#!type manifests/deploy-app.yaml #use this on Windows
!cat manifests/deploy-app.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: deploy-nginx
  labels:
    app: nginx
    env: prod
spec:
  replicas: 3
  revisionHistoryLimit: 3
  selector:
    matchLabels:
      app: nginx
      env: prod
  template:
    metadata:
      labels:
        app: nginx
        env: prod
    spec:
      containers:
      - name: nginx
        image: nginx:alpine
        ports:
        - containerPort: 80
        resources:
          requests:
            cpu: 100m
            memory: 128Mi
          limits:
            cpu: 250m
            memory: 256Mi

## Deploy a Service

- The Service definition is in the YAML file `manifests/loadbalancer.yaml`.

In [7]:
!kubectl apply -f manifests/loadbalancer.yaml

service/svc-example created


## Let's look at the Service's YAML

**Note:**

- The service's name is `svc-example`.
- The service's is:
  - Of type `LoadBalancer` which means it will be assigned an external (public) IP address.
  - Listening on `port` 8080.
  - Redirecting traffic to `targetPort` 80.
- The servie's `selector` defines two labels (that match the Pod template's labels for the Deployment):
  - `app: nginx` and `env: prod`
  - This means the Service will (round robin) load balance between the three Pods in the Deployment's ReplicaSet.

```bash
apiVersion: v1
kind: Service
metadata:
  name: svc-example    # the Service's name is svc-example
spec:
  type: LoadBalancer   # the Service's type is LoadBalancer
  selector:
    app: nginx         # the Service's selector defines two labels (the Service's selector matches the Pod template's two labels):
    env: prod          # app = nginx and env = prod
  ports:
  - protocol: TCP      # the Service's protocol is TCP
    port: 8080         # the Service is listening for internal (private) traffic on port 8080
    targetPort: 80     # the Service is redirecting traffic to targetPort 80
```

In [8]:
#!type manifests/loadbalancer.yaml #use this on Windows
!cat manifests/loadbalancer.yaml

apiVersion: v1
kind: Service
metadata:
  name: svc-example
spec:
  type: LoadBalancer
  selector:
    app: nginx
    env: prod
  ports:
  - protocol: TCP
    port: 8080
    targetPort: 80

## Get Pods

- We see that 3 Pods are running.

In [9]:
#!kubectl get po -o wide
!kubectl get pods -o wide

NAME                            READY   STATUS    RESTARTS   AGE   IP            NODE           NOMINATED NODE   READINESS GATES
deploy-nginx-7b944585d7-gpslf   1/1     Running   0          25s   10.244.2.8    minikube-m03   <none>           <none>
deploy-nginx-7b944585d7-l5dr7   1/1     Running   0          25s   10.244.0.10   minikube       <none>           <none>
deploy-nginx-7b944585d7-xl8kd   1/1     Running   0          25s   10.244.1.10   minikube-m02   <none>           <none>


## Get Services

- Notice the `TYPE` is `LoadBalancer` for the `svc-example` Service.
- Notice the `EXTERNAL-IP` and `PORT` 8080 used by the `Loadbalancer`.

In [10]:
#!kubectl get svc -o wide
!kubectl get services -o wide

NAME          TYPE           CLUSTER-IP       EXTERNAL-IP      PORT(S)          AGE   SELECTOR
kubernetes    ClusterIP      10.96.0.1        <none>           443/TCP          11h   <none>
svc-example   LoadBalancer   10.100.112.197   192.168.49.200   8080:32469/TCP   16s   app=nginx,env=prod


## Access the Load Balancer Service

- Use the Load Balancer Service's assigned `EXTERNAL-IP` address above and the Service's port number 8080: `http://EXTERNAL-IP:8080`
- When using a cloud provider, you would use the cloud provider's LoadBalancer external IP address and your Service's port number.

In [11]:
!curl http://192.168.49.200:8080

<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
html { color-scheme: light dark; }
body { width: 35em; margin: 0 auto;
font-family: Tahoma, Verdana, Arial, sans-serif; }
</style>
</head>
<body>
<h1>Welcome to nginx!</h1>
<p>If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="http://nginx.org/">nginx.org</a>.<br/>
Commercial support is available at
<a href="http://nginx.com/">nginx.com</a>.</p>

<p><em>Thank you for using nginx.</em></p>
</body>
</html>


## Delete the Service and the Deployment

In [12]:
!kubectl delete -f manifests/loadbalancer.yaml
!kubectl delete -f manifests/deploy-app.yaml

service "svc-example" deleted
deployment.apps "deploy-nginx" deleted


## List Services, Deployments and Pods

- We see that the Service and Deployment with associated Pods were deleted.

In [13]:
#!kubectl get svc
#!kubectl get deploy
#!kubectl get po
!kubectl get services
!kubectl get deployments
!kubectl get pods

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   11h
No resources found in default namespace.
No resources found in default namespace.


## Disable the Metallb addon

In [14]:
!minikube addons disable metallb

🌑  "The 'metallb' addon is disabled
